# Benchmark Torch reimplementation and meta's Fasttext 

We use the same model in both cases.
Hyperparams set : 
- embedding_dim=80
- num_tokens=10000

## Training

General params

In [1]:
!pip install -r ../requirements.txt -q

In [35]:
!pip install fasttext -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.13.6-py3-none-any.whl.metadata (9.5 kB)
Using cached pybind11-2.13.6-py3-none-any.whl (243 kB)
  Created wheel for fasttext: filename=fasttext-0.9.3-cp312-cp312-linux_x86_64.whl size=326785 sha256=a4bfe64e723afda200abd297fddb350bbc8094165c81851c8eb3beef1b707df0
  Stored in directory: /home/onyxia/.cache/pip/wheels/20/27/95/a7baf1b435f1cbde017cabdf1e9688526d2b0e929255a359c6
Successfully built fasttext


In [37]:
import os
import sys
import time
import numpy as np
import pandas as pd
import pyarrow.parquet as pq
import s3fs
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import fasttext
sys.path.append("../")
from torchFastText import torchFastText
from torchFastText.preprocess import clean_text_feature
from torchFastText.datasets import NGramTokenizer
sys.path.append("./notebooks")

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Hyperparamètres

In [21]:
params={
    "y_name": "nace",
    "text_feature": "text",
    "df_sample_size": 100000,
    "max_epochs": 10,
    "train_proportion": 0.8,
    "lr": 0.001,
    "buckets": 10000,
    "dim": 80,
    "minCount": 1,
    "minn": 3,
    "maxn": 6,
    "wordNgrams": 3,
    "ft_thread": 100,
    "ft_loss": "ova", #"softmax",
    "ft_lrUpdateRate": 0, #100
    "ft_neg": 5, # 5
    "torch_batch_size": 256,
    "torch_patience": 3,
    "torch_sparse": False,
    "torch_num_workers": 100,
    "torch_categorical_features": [] ,
}

sparse=False
factor=0.5
min_lr=1e-06
start_year=2018
textual_features=['NAT_LIB', 'AGRI']
tokenizer_name=NGramTokenizer
experiment_name = model_comparison_s4
number_of_training_observations = 2460521
trainer_name = Lightning
patience_early_stopping = 3
loss_name = CrossEntropyLoss
revision = NAF2008
name = torchFastText
dataset = FastTextModelDataset
categorical_embedding_dims = 5
optimizer_name = Adam
scheduler_name = ReduceLROnPlateau
patience = 2
start_month = 1
text_feature = libelle
categorical_features = ['TYP', 'NAT', 'SRF', 'CJ', 'CRT']
max_n = 6
num_tokens = 100000
output_dir = ./outputs
epochs = 10
betas = (0.9, 0.999)
eps = 1e-08
weight_decay = 0
amsgrad = False
maximize = False
foreach = None
capturable = False
differentiable = False
fused = None
direct_bagging = True
num_epochs = 10
batch_size = 256
sirene = sirene_4
min_count = 100

Data

In [3]:
from utils import add_libelles, clean_and_tokenize_df, stratified_split_rare_labels

In [4]:
fs = s3fs.S3FileSystem(
    client_kwargs={"endpoint_url": "https://minio.lab.sspcloud.fr"},
    anon=True,
)
df = (
    pq.ParquetDataset(
        "projet-ape/extractions/20241027_sirene4.parquet",
        filesystem=fs,
    )
    .read_pandas()
    .to_pandas()
).sample(frac=0.001).fillna(np.nan)
print(f"df contains {df.shape[0]} rows")

with fs.open("projet-ape/data/naf2008.csv") as file:
    naf2008 = pd.read_csv(file, sep=";")

categorical_features = ["evenement_type", "cj",  "activ_nat_et", "liasse_type", "activ_surf_et", "activ_perm_et"]
text_feature = "libelle"
y = "apet_finale"
textual_features = None


2025-03-05 15:53:13 - botocore.httpchecksum - Skipping checksum validation. Response did not contain one of the following algorithms: ['crc32', 'sha1', 'sha256'].
2025-03-05 15:53:13 - botocore.httpchecksum - Skipping checksum validation. Response did not contain one of the following algorithms: ['crc32', 'sha1', 'sha256'].
2025-03-05 15:53:13 - botocore.httpchecksum - Skipping checksum validation. Response did not contain one of the following algorithms: ['crc32', 'sha1', 'sha256'].
2025-03-05 15:53:14 - botocore.httpchecksum - Skipping checksum validation. Response did not contain one of the following algorithms: ['crc32', 'sha1', 'sha256'].
2025-03-05 15:53:14 - botocore.httpchecksum - Skipping checksum validation. Response did not contain one of the following algorithms: ['crc32', 'sha1', 'sha256'].
2025-03-05 15:53:17 - botocore.httpchecksum - Skipping checksum validation. Response did not contain one of the following algorithms: ['crc32', 'sha1', 'sha256'].


df contains 2645 rows


In [5]:

df = add_libelles(df, naf2008, y, text_feature, textual_features, categorical_features)
df["libelle_processed"] = clean_text_feature(df["libelle"])
df

	*** 732 codes have been added in the database...



,liasse_numero,id,evenement_type,categorie_demande,cj,domas,ssdom,liasse_type,activ_sec_agri_et,activ_nat_et,...,date_modification,mode_calcul_apet,mode_calcul_apen,apet_finale,codif_apet,codif_apen,emetteur,libelle,is_apet,libelle_processed
320914,J00019262054,616119.0,01M,CG,5499,NaN,NaN,C,NaN,99,...,2023-04-14 13:53:04.980409+00:00,AUTO,NaN,8559A,True,False,Guichet unique,Formateur langue étrangère,1.0,formateur langu etranger
707376,J00032127136,1519642.0,01P,CG,NaN,NaN,NaN,L,NaN,NaN,...,2023-09-04 14:32:53.562975+00:00,FASTTEXT,NaN,7500Z,True,False,Guichet unique,Vétérinaire collaborateur libéral sans structure,1.0,veterinair collabor liberal san structur
357930,J00019509561,699684.0,01P,CG,NaN,NaN,NaN,M,NaN,99,...,2023-04-27 00:34:37.455876+00:00,AUTO,NaN,9609Z,True,False,Guichet unique,Pension et garde d'animaux de compagnie d'espè...,1.0,pension gard d'animal compagn d'espec domestiq...
974088,J00044172534,2199945.0,01P,CG,NaN,NaN,NaN,M,NaN,14,...,2023-11-27 16:40:17.405710+00:00,AUTO,NaN,4334Z,True,False,Guichet unique,Travaux de peinture intérieur de bâtiment,1.0,traval peintur interieur bat
471916,J00023167000,955693.0,01M,CG,6540,NaN,NaN,G,NaN,NaN,...,2023-06-07 07:24:04.644934+00:00,AUTO,NaN,6820B,True,False,Guichet unique,propriété immobilière,1.0,propriet immobilier
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
727,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaT,NaN,NaN,9609Z,NaN,NaN,NaN,Autres services personnels n.c.a.,NaN,autr servic personnel n.c.a.
728,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaT,NaN,NaN,9700Z,NaN,NaN,NaN,Activités des ménages en tant qu'employeurs de...,NaN,activit menag tant qu'employeur personnel domest
729,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaT,NaN,NaN,9810Z,NaN,NaN,NaN,Activités indifférenciées des ménages en tant ...,NaN,activit indifferencie menag tant producteur bi...
730,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaT,NaN,NaN,9820Z,NaN,NaN,NaN,Activités indifférenciées des ménages en tant ...,NaN,activit indifferencie menag tant producteur se...


In [6]:

encoder = LabelEncoder()
df["apet_finale"] = encoder.fit_transform(df["apet_finale"])

df, _ = clean_and_tokenize_df(df, text_feature="libelle_processed")
X = df[["libelle_processed", "EVT", "CJ", "NAT", "TYP", "CRT", "SRF"]].values
y = df["apet_finale"].values

df

/home/onyxia/work/torch-fastText/notebooks/utils.py:60: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna("nan", inplace=True)


,libelle_processed,EVT,CJ,NAT,TYP,SRF,CRT,apet_finale
320914,formateur langu etranger,1,12,6,1,0,1,660
707376,veterinair collabor liberal san structur,2,28,7,5,0,1,604
357930,pension gard d'animal compagn d'espec domestiq...,2,28,6,6,0,1,727
974088,traval peintur interieur bat,2,28,3,6,0,1,359
471916,propriet immobilier,1,21,7,3,0,0,579
...,...,...,...,...,...,...,...,...
727,autr servic personnel n.c.a.,23,28,7,7,0,0,727
728,activit menag tant qu'employeur personnel domest,23,28,7,7,0,0,728
729,activit indifferencie menag tant producteur bi...,23,28,7,7,0,0,729
730,activit indifferencie menag tant producteur se...,23,28,7,7,0,0,730


In [7]:
X_train, X_test, y_train, y_test = stratified_split_rare_labels(X, y)
assert set(range(len(naf2008["code"]))) == set(np.unique(y_train))
X_train

array([['cultur riz', 23, 28, ..., 7, 0, 0],
       ['cultur cann sucr', 23, 28, ..., 7, 0, 0],
       ['cultur tabac', 23, 28, ..., 7, 0, 0],
       ...,
       ["auteur d'oeuvr cinematograph audiovisuelles; concepteur createur photographique, pris vue, reportage; auteurs-compositeur",
        2, 28, ..., 5, 1, 0],
       ['particuli producteur electricit', 2, 28, ..., 4, 1, 0],
       ["particuli producteur d'energ photovolta", 2, 28, ..., 4, 1, 0]],
      dtype=object)

### TorchFasttext

### Fasttext meta

In [8]:
df_train = pd.concat([pd.DataFrame(X_train, columns=["libelle_processed", "EVT", "CJ", "NAT", "TYP", "CRT", "SRF"]), pd.DataFrame(y_train, columns=["apet_finale"])], axis=1)
df_test = pd.concat([pd.DataFrame(X_test, columns=["libelle_processed", "EVT", "CJ", "NAT", "TYP", "CRT", "SRF"]), pd.DataFrame(y_test, columns=["apet_finale"])], axis=1)

In [9]:
df_train

,libelle_processed,EVT,CJ,NAT,TYP,CRT,SRF,apet_finale
0,cultur riz,23,28,7,7,0,0,1
1,cultur cann sucr,23,28,7,7,0,0,3
2,cultur tabac,23,28,7,7,0,0,4
3,cultur plant fibr,23,28,7,7,0,0,5
4,cultur fruit tropical subtropical,23,28,7,7,0,0,8
...,...,...,...,...,...,...,...,...
2788,install mainten system electr,2,28,3,6,1,0,348
2789,livraison rep domicil velo,2,28,7,10,1,0,513
2790,auteur d'oeuvr cinematograph audiovisuelles; c...,2,28,7,5,1,0,536
2791,particuli producteur electricit,2,28,7,4,1,0,313


In [14]:
from typing import List, Optional, Dict
from pathlib import Path
def write_training_data(
    df: pd.DataFrame,
    y: str,
    text_feature: str,
    categorical_features: Optional[List[str]],
    label_prefix: str = "__label__",
) -> str:
    """
    Write training data to file.

    Args:
        df (pd.DataFrame): DataFrame.
        y (str): Output variable name.
        text_feature (str): Text feature.
        categorical_features (Optional[List[str]]): Categorical features.
        label_prefix (str, optional): Label prefix. Defaults to "__label__".

    Returns:
        str: Training data path.
    """
    training_data_path = Path("data/training_data.txt")

    with open(training_data_path, "w", encoding="utf-8") as file:
        for _, item in df.iterrows():
            formatted_item = f"{label_prefix}{item[y]} {item[text_feature]}"
            if categorical_features != []:
                for feature in categorical_features:
                    formatted_item += f" {feature}_{item[feature]}"
            file.write(f"{formatted_item}\n")
    return training_data_path.as_posix()

# Write training data in a .txt file (fasttext-specific)
training_data_path = write_training_data(
    df=df_train,
    y="apet_finale",
    text_feature="libelle_processed",
    categorical_features=["EVT", "CJ", "NAT", "TYP", "CRT", "SRF"],
)

In [31]:
params["ft_loss"]

'ova'

In [38]:
# Train the fasttext model

start_time = time.time()

model_ft = fasttext.train_supervised(
    input=training_data_path,
    dim=params["dim"],
    lr=params["lr"],
    epoch=params["max_epochs"],
    lrUpdateRate=params["ft_lrUpdateRate"],
    neg=params["ft_neg"],
    wordNgrams=params["wordNgrams"],
    minn=params["minn"],
    maxn=params["maxn"],
    minCount=params["minCount"],
    bucket=params["buckets"],
    thread=params["ft_thread"],
    loss=params["ft_loss"],
    label_prefix="__label__",
    verbose=2
)

end_time = time.time()
elapsed_time_ft = (end_time - start_time) / 60
print("Temps écoulé pour entrainer la lib fasttext : ", elapsed_time_ft, " minutes")

Read 0M words
Number of words:  4086
Number of labels: 732
Progress: 100.3% words/sec/thread:    2835 lr: -0.000003 avg.loss: 381.136841 ETA:   0h 0m 0s

Temps écoulé pour entrainer la lib fasttext :  0.024307398001352946  minutes


Progress: 100.0% words/sec/thread:    2831 lr:  0.000000 avg.loss: 381.136841 ETA:   0h 0m 0s
